Сгенерируем синтетические данные для регрессии

In [1]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [20]:
# dataset for linear regression with random parameters generation

# features = np.random.randint(5, 51)
features = 5
informative = int(features * np.random.random_sample())
noise = np.random.random_sample()
bias = np.random.randint(0, 50)

X, y = make_regression(n_samples=100, n_features=features, n_informative=informative, bias=bias, noise=noise, random_state=42)

print(f'features = {features}, informative = {informative}, noise = {noise}, bias = {bias}')
print(X.shape, y.shape)

features = 5, informative = 2, noise = 0.4562629206171043, bias = 24
(100, 5) (100,)


In [21]:
X

array([[-1.32818605, -1.22084365, -1.95967012,  0.2088636 ,  0.19686124],
       [-1.24573878, -0.44651495,  0.21409374,  0.85639879,  0.17318093],
       [ 0.04698059, -1.59442766,  0.0052437 , -0.59937502, -0.45006547],
       [-1.91328024, -0.46341769,  0.24196227, -0.46572975, -1.72491783],
       [-0.55364931, -0.79287283, -1.03524232, -0.10703036, -1.19787789],
       [ 0.95927083,  0.02451017,  1.45114361,  0.49799829,  2.15318246],
       [ 0.09176078,  0.8219025 , -0.29900735,  0.08704707, -1.98756891],
       [ 0.28099187, -1.9520878 ,  0.58831721, -0.1517851 , -0.62269952],
       [ 0.8496021 , -0.20812225, -0.58936476, -0.49300093,  0.35701549],
       [-0.48536355,  0.75896922, -0.23681861, -0.77282521,  0.08187414],
       [ 0.36163603,  1.35624003,  1.0035329 , -0.07201012, -0.64511975],
       [-1.02438764, -0.92693047, -3.24126734, -0.05952536, -0.25256815],
       [ 0.86575519, -0.79252074,  0.50498728, -0.11473644, -1.20029641],
       [-0.90802408, -0.56228753,  0.3

In [22]:
y

array([ -43.31404711,   44.42150991, -112.90914253,  -32.81265577,
        -33.16984635,   54.11531864,   81.61053542, -110.94436016,
        -17.45018523,   28.54835018,  107.83635728,  -39.11557992,
        -33.69968035,  -70.3515812 ,   65.01259543,   -6.52090629,
         78.99393074,  145.40932562,   83.80062972,   34.91595361,
        -24.78588925, -168.44480451,   74.68744122, -104.27634699,
         40.67268946,  158.01808359,   84.79687311,    3.13873429,
          4.32952609,    5.67130337,   65.15110894,   30.23954648,
         53.74273617,   97.60526846,  153.39114183,   44.18264443,
         90.88089131,   -0.81538112,  121.35315232,   11.21596777,
         67.19163812,   22.44566706,  -80.4125265 ,  -28.85794606,
         -5.79803017,  -48.81116199,  -50.92039867,  128.44135258,
         64.30360745,  -61.12267359,  135.35950559,  -65.12573746,
        -42.90011082,   69.30523732,   64.09488076,   63.30548776,
         36.3901044 ,   40.37637983,   73.20346402, -104.59864

In [23]:
class NaiveMLPRegressor:
    def __init__(self, hidden_layer_sizes=(100,), learning_rate=0.001, max_iter=10):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.weights = []
        self.biases = []

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _weights_initialization(self, input_size, output_size):
        # инициализируем hidden_layer_sizes=(100,) разных матриц весов и смещений
        for i in range(len(self.hidden_layer_sizes)):
            self.weights.append(np.random.normal(0, 0.1, size=input_size))
            self.biases.append([np.random.normal(0, 0.1)])

    def train(self, X_train, y_train):
        input_dim = X_train.shape[1]
        output_dim = 1
        layers_output = []

        self._weights_initialization(input_dim, output_dim)

        for i in range(self.max_iter):
            # forward
            for j in range(len(self.weights)):
                neuron_output = np.dot(X_train, self.weights[j]) + self.biases[j]
                after_activation = self._sigmoid(neuron_output)
                layers_output.append(after_activation)

            error = y_train - layers_output[-1]

            # backward
            for k in range(len(self.weights) - 1, -1, -1):
                if k == len(self.weights) - 1:
                    output_error = error * layers_output[k] * (1 - layers_output[k])
                    self.weights[k] += self.learning_rate * np.dot(X_train.T, output_error)
                    self.biases[k] += self.learning_rate * np.sum(output_error)
                else:
                    hidden_error = np.dot(output_error, self.weights[k + 1].T) * layers_output[k] * (1 - layers_output[k])
                    self.weights[k] += self.learning_rate * np.dot(X_train.T, hidden_error)
                    self.biases[k] += self.learning_rate * np.sum(hidden_error)

            return self

    def predict(self, X):
        for i in range(len(self.weights)):
            neuron_output = np.dot(X, self.weights[i]) + self.biases[i]
            y_pred = self._sigmoid(neuron_output)
        return y_pred

In [24]:
naive_MLP = NaiveMLPRegressor()
naive_MLP.train(X, y)
y_pred = naive_MLP.predict(X)

In [25]:
print(f'mse = {mean_squared_error(y, y_pred)}, mae = {mean_absolute_error(y, y_pred)}', end='\n\n')

mse = 5995.818893586923, mae = 63.67175758341165



**Для сравнения**

In [26]:
sklearnMLP = MLPRegressor(activation='logistic', max_iter=10)
sklearnMLP.fit(X, y)
y_pred = sklearnMLP.predict(X)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


In [27]:
print(f'mse = {mean_squared_error(y, y_pred)}, mae = {mean_absolute_error(y, y_pred)}', end='\n\n')

mse = 6069.288971376356, mae = 64.07635967165844

